In [1]:
# Imports
from Gibbs.GibbsSampler import GibbsSampler
from Gibbs.Parameters import *
from DataGeneration import generate_multiple_series
import Gibbs.Initialize as Initialize
import Gibbs.Priors as Priors
import numpy as np
from Trends import Regressors
from Trends import LowFrequencyTrends

## Load Data
Load data and populations weights for the Gibbs sampler

In [2]:
data = generate_multiple_series(150, 120, 10**4, 10**5, "gibbs_data.csv")
q = 16
q_hat = 31
n = len(data)
m = 25
l = 10
T = len(data[0])
w = np.random.random(n)
R_hat = Regressors.find_regressors(T, q_hat).T
# Normalize the weights to sum to 1
w = w/np.sum(w)
print('Sum of weights: '+str(np.sum(w)))
print('Dimensions of R_hat: '+str(R_hat.shape))

Sum of weights: 1.0
Dimensions of R_hat: (150, 32)


## Initialize the Gibbs Sampler

### Step-by-step initialization

Initialize $X_i$:

In [3]:
X_i = Initialize.initialize_X(data, q)
print("# Entries in X_i: "+str(len(X_i)))
print("Length of each coefficient vector: "+str(len(X_i[0])))

# Entries in X_i: 120
Length of each coefficient vector: 32


Initialize $(F, S_m)$:

In [4]:
F = Initialize.initialize_F(X_i, w, q_hat)
print("Length of F: "+str(len(F)))

Length of F: 32


In [5]:
S_m, sigma_m, Sigma_m = Initialize.initialize_S_m(T, R_hat)
print("Length of S_m: "+str(len(S_m)))

Length of S_m: 32


Initialize $\{K(j)\}_{j=1}^{25}$ and $\{J(i)\}_{i=1}^n$:

In [6]:
K = Priors.group_factors(m, l)
J = Priors.group_factors(n, m) 
print("K has "+str(len(K))+" factors")
print("J has "+str(len(J))+" factors")

K has 25 factors
J has 120 factors


Initalize $\{\lambda_{c,i}\}_{i=1}^n$ and $\{\lambda_{g,j}\}_{j=1}^{25}$ parameters:

In [7]:
lambdas = Lambda_Parameters(n, m)
print("We have "+ str(len(lambdas.lambda_c_i))+" lambda_c_i parameters")
print("We have "+ str(len(lambdas.lambda_g_j))+" lambda_g_j parameters")

We have 120 lambda_c_i parameters
We have 25 lambda_g_j parameters


Initalize $\{\kappa_{c,i}\}_{i=1}^n$, $\{\kappa_{g,j}\}_{j=1}^{25}$ 
and $\{\kappa_{h,k}\}_{k=1}^{10}$ parameters:

In [8]:
kappas = Kappa_Parameters(n,m,l)
print("We have "+ str(len(kappas.kappa_c_i))+" kappa_c_i parameters")
print("We have "+ str(len(kappas.kappa_g_j))+" kappa_g_j parameters")
print("We have "+ str(len(kappas.kappa_h_k))+" kappa_h_k parameters")

We have 120 kappa_c_i parameters
We have 25 kappa_g_j parameters
We have 10 kappa_h_k parameters


Initialize all $p$ parameters

In [9]:
p_parameters = P_Parameters()
print("We have "+ str(len(p_parameters.p_c_lambda))+" p_c_l parameters")
print("We have "+ str(len(p_parameters.p_g_lambda))+" p_g_l parameters")
print("We have "+ str(len(p_parameters.p_c_theta))+" p_c_theta parameters")
print("We have "+ str(len(p_parameters.p_g_theta))+" p_g_theta parameters")
print("We have "+ str(len(p_parameters.p_h_theta))+" p_h_theta parameters")
print("We have "+ str(len(p_parameters.p_c_kappa))+" p_c_k parameters")
print("We have "+ str(len(p_parameters.p_g_kappa))+" p_g_k parameters")
print("We have "+ str(len(p_parameters.p_h_k))+" p_h_k parameters")

We have 25 p_c_l parameters
We have 25 p_g_l parameters
We have 100 p_c_theta parameters
We have 100 p_g_theta parameters
We have 100 p_h_theta parameters
We have 25 p_c_k parameters
We have 25 p_g_k parameters
We have 25 p_h_k parameters


Initialize $\sigma_{\Delta a}^2$:

In [10]:
s_Da = Initialize.inverse_squared(0.03**2)
print("s_Da = "+str(s_Da))

s_Da = 201.07201072010722


Initialize $f_0$, $\mu_m$ and $\mu_c$:

In [11]:
f_0 = Priors.flat_prior(0, 10**6)
mu_m = Priors.flat_prior(0, 10**6)
mu_c = Priors.flat_prior(0, 10**6)
print("f_0 = "+str(f_0))
print("mu_m = "+str(mu_m))
print("mu_c = "+str(mu_c))

f_0 = 102962.60741261354
mu_m = 354389.87821151677
mu_c = 810274.2220690281


Initialize $\omega^2$:

In [12]:
omega_squared = Initialize.inverse_squared(1)
print("omega_squared = " + str(omega_squared))

omega_squared = 279.8727987279873


Initialize $U_{c,i}$, $U_{g,j}$ and $U_{h,k}$:

In [13]:
us = U_Parameters(omega_squared, lambdas, kappas, R_hat, n, m, l, T)

c:\Users\konto\Documents\Economy-and-Population-Growth\Gibbs\Priors.py:78: RuntimeWarning: covariance is not positive-semidefinite.
  return np.random.multivariate_normal(mean=mean, cov=cov, size=n)


In [14]:
print(len(us.U_c))
print(len(us.U_c[0]))
print(len(us.U_g))
print(len(us.U_g[0]))
print(len(us.H))
print(len(us.H[0]))

120
32
25
32
10
32


In [15]:
G = []
for j in range(m):
    G_j = lambdas.lambda_g_j[j]*us.H[K[j]]+us.U_g[j]
    G.append(G_j)

C = []
i_1 = np.zeros(q_hat+1)
i_1[0] = 1
for i in range(n):
    C_i = mu_c*i_1+lambdas.lambda_c_i[i]*G[J[i]]+us.U_c[i]
    C.append(C_i)
G = np.array(G)
C = np.array(C)
print(len(G))
print(len(C))

25
120


## Steps of a Gibbs Draw

### Step 1: Calculate $\{X_i\}_{i=1}^n$

Calculate $Y^0$, $\mu_C$, $\Sigma$, $\Beta$, $w$, $m$ and $V$:

In [16]:
Y0 = w@C

In [17]:
i_1 = np.zeros(q_hat+1)
i_1[0] = 1
mu_C = np.concatenate([mu_c*i_1.T+lambdas.lambda_c_i[i]*G[J[i]].T for i in range(n)]).T

In [18]:
dim = (q_hat+1)*n
Sigma = np.zeros((dim, dim))
Sigma[0:(q_hat+1), 0:(q_hat+1)] = us.S_U_c[0]
for i in range(n):
    Sigma[i*(q_hat+1):(i+1)*(q_hat+1), i*(q_hat+1):(i+1)*(q_hat+1)] = us.S_U_c[i]

In [19]:
# X = np.concatenate([X_i[i].T for i in range(n)]).T
# X = np.reshape(X_i, (dim, ))
X = X_i.flatten()
Ys = []
for i in range(n):
    Y_i, _ = LowFrequencyTrends.find_trends(data[i], q)
    Ys.append(Y_i)

Y = np.concatenate([Ys[i] for i in range(len(Ys))])

def find_B(A, C):
    a = len(A)
    c = len(C)
    B = np.zeros((a, c))
    for i in range(a):
        B[i, :] = A[i]*C
    return B.T

B = find_B(Y, X)
I = np.identity(q_hat+1)
ws = np.kron(w.T, I).T
print(B.shape)

(3840, 2040)


In [20]:
Cs = np.concatenate([C[i] for i in range(len(C))])

In [21]:
help = Sigma@B
mm = help@np.linalg.inv(B.T@help)@B.T@(Cs-mu_C)

In [35]:
V = Sigma-help@np.linalg.inv(B.T@help)@B.T@Sigma
print(V.shape)

(3840, 3840)


Draw $Z_0$, calculate $Z_1$, draw $\epsilon$ and caclulate $C$:

In [23]:
Z0 = Priors.multivariate_normal_prior(mu_C, V/np.linalg.norm(V))[0]

In [24]:
print(Z0)

[ 8.07442210e+05  1.85258891e+01 -2.38149614e+03 ...  9.48466524e+01
 -7.43887959e+01  5.51683620e+01]


In [25]:
Z1 = Z0-help@np.linalg.inv(B.T@help)@B.T@(Z0-Cs)
print(Z1.shape)

(3840,)


In [26]:
Delta = np.identity(q_hat+1)*0.01**2
epsilon = Priors.multivariate_normal_prior(Y0, Delta)[0]
e = np.repeat(epsilon, n)

In [27]:
Cs = Z1 - V@ws@(ws.T@V@ws+Delta)@ws.T@(Z1-e)
print(Cs.shape)
CC = np.reshape(Cs, (n, q_hat+1))
print(CC.shape)
XX = CC+F
print(X_i.shape)
print(XX.shape)

(3840,)
(120, 32)
(120, 32)
(120, 32)


### Step 2: Calculate $(F, S_m)$:

Calculate $\mu_F$ and $\Sigma_F$:

In [36]:
i_2 = np.zeros(q_hat+1)
i_2[1] = 1

mu_F = i_1*f_0+i_2*mu_m
print(mu_F.shape)

(32,)


In [37]:
A, Sigma_A = Initialize.init_Sigma_A(R_hat, T, q_hat, s_Da)
Sigma_F = sigma_m**3*Sigma_m+s_Da**2*Sigma_A
print(Sigma_F.shape)

(32, 32)


c:\Users\konto\Documents\Economy-and-Population-Growth\Gibbs\Priors.py:78: RuntimeWarning: covariance is not positive-semidefinite.
  return np.random.multivariate_normal(mean=mean, cov=cov, size=n)


Calculate $e$:

In [30]:
e = np.kron(np.ones(n).T, I).T
print(e.shape)

(3840, 32)


Calculate $m_F$, $V_F$, and draw $F-\mu_F$:

In [38]:
V_F = np.linalg.inv(np.linalg.inv(Sigma_F)+e.T@np.linalg.inv(Sigma)@e+np.linalg.inv(Delta))
print(V_F.shape)

(32, 32)


In [32]:
m_F = V_F@(e.T@np.linalg.inv(Sigma)@(X-mu_C-e@mu_F)+np.linalg.inv(Delta)@((ws.T@X)-Y0-mu_F))
print(m_F.shape)

(32,)


In [33]:
draw = Priors.multivariate_normal_prior(m_F, V_F)[0]
FF = draw+mu_F
print(FF.shape)

(32,)


Calculate $\Sigma_S$ and draw $S_m$:

In [39]:
Sigma_S = sigma_m**2*Sigma_m
print(Sigma_S.shape)
mean_S_m = Sigma_S@np.linalg.inv(Sigma_F)@(FF-mu_F)
var_S_m = Sigma_S-Sigma_S@np.linalg.inv(Sigma_F)@Sigma_S

SS_m = Priors.multivariate_normal_prior(mean_S_m, var_S_m)[0]

(32, 32)
